In [ ]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-2j617h9r/unsloth_7c16e3cebb7540dbbe4ba4d6c1d967e0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-2j617h9r/unsloth_7c16e3cebb7540dbbe4ba4d6c1d967e0
  Resolved https://github.com/unslothai/unsloth.git to commit 3e4c5a323c16bbda2c92212b790073c4e99c2a55
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import torch
from unsloth import FastLanguageModel

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:71: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


In [ ]:
major_version, minor_version = torch.cuda.get_device_capability()

In [ ]:
import torch
max_seq_length = 4096
dtype = None
load_in_4bit = True
model_name = "unsloth/tinyllama-bnb-4bit"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.2
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:

model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = False,
    random_state = 3407,
    max_seq_length = max_seq_length,
)

Unsloth 2024.2 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging


In [ ]:
logging.set_verbosity_info()

In [ ]:
DATASET_ID = 'knowrohit07/know_medical_dialogue_v2'

In [ ]:
# get model text generate
def get_model_generate(index: int):

    medical_condition = test_dataset['instruction'][index]
    treatment_options = test_dataset['output'][index]

    prompt = f'''
    ### Instruction:
    For describe the treatment options the following conversation.

    ### Explaining medical conditions:
    {medical_condition}

    ### Describe the treatment options:
    '''

    input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=500,
        )

    output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

    dash_line = '-'.join('' for x in range(100))
    result = f''''{dash_line}\nINPUT PROMPT:\n{prompt}{dash_line}\nBASELINE HUMAN DESCRIBE THE TREATMENT OPTIONS:\n{treatment_options}\n{dash_line}\nTRAINED MODEL GENERATED TEXT :\n{output}'''

    return result

In [ ]:
def format_instruction(medical_condition: str, treatment_options: str):
	return f'''### Instruction:
            For describe the treatment options the following conversation.

            ### Explaining medical conditions:
            {medical_condition.strip()}

            ### Describe the treatment options:
            {treatment_options.strip()}
            '''.strip()

In [ ]:
from datasets import Dataset, load_dataset, DatasetDict
dataset = load_dataset(DATASET_ID)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
logging.set_verbosity_info()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,013 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 376
 "-____-"     Number of trainable parameters = 25,231,360


Step,Training Loss
1,2.375600
2,2.299000
3,2.399000
4,2.407500
5,2.346700
6,2.366600
7,2.366600
8,2.389400
9,2.362100
10,2.362300




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe fibonacci sequence is a sequence of numbers that can be generated by starting with 1 and then adding the previous number to the previous number. The sequence is: 1, 1, 2, 3, 5, 8, 13, 22, 44,']

In [ ]:
input = "My daughter ( F, 18 y/o, 5'5', 165lbs) has been feeling poorly for a few weeks. \
She had COVID a couple of months ago and symptoms have are much worse in the last month or so. \
She feels light headed, breathless, dizzy, and her heart rate (HR) goes from ~65 lying down to ~250 when standing. \
Today she tells me HR has been around 195 all day and she feels really lousy. \
What should I do?"

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Label what the parent should do to take care of their daughter.", # instruction
        input, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nLabel what the parent should do to take care of their daughter.\n\n### Input:\nMy daughter ( F, 18 y/o, 5'5', 165lbs) has been feeling poorly for a few weeks. She had COVID a couple of months ago and symptoms have are much worse in the last month or so. She feels light headed, breathless, dizzy, and her heart rate (HR) goes from ~65 lying down to ~250 when standing. Today she tells me HR has been around 195 all day and she feels really lousy. What should I do?\n\n### Response:\nThe parent should take care of their daughter by providing her with the necessary medical attention and support. They should make sure that she gets enough rest, eats a healthy diet, and gets plenty of exercise. They should also make sure that she gets enough sleep, and that she has access to a reliable source of"]